# Packages


In [49]:
import spacy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
from tqdm import tqdm
import json
import duden

from collections import Counter

import pandas as pd

# Get Lyrics


- API Schlüssel und Zugangsdaten einsetzen
- Playlist IDs von Spotify suchen 
- Jeder Track der Playlist wird auf LyricsGenius gesucht
- Sollte zu einem Song kein Lyrics gefunden werden, wird die Anzahl hochgezählt
- Es wird eine JSON Datei gebildet

In [6]:
SPOTIPY_CLIENT_ID = '46da6046625f409cb53ab06bf807861d'
SPOTIPY_CLIENT_SECRET = '00efb0ab68db4f1eaf27ec53a4c1e068'
GENIUS_ACCESS_TOKEN = 'OsBvEsJ_OLKYR42CJoaA_N97GNOS44XXFk4KPPcLtCYnWmDwnnzY2RpcG_SdG25a'

In [ ]:
playlist_id = '2siI2ILJEPr88yibWLgQQl'  #Playlist: "Deutsche Songs die jeder kennt ;)"

Ausführen von getlyrics.py

## Analysis

In [31]:
with open("./all_songs.json") as f:
    songs_dict = json.load(f)

1. Spacy

Ausführen von cyspa.py

In [32]:
#NEU #Spacy muss gedownloaded werden mit : pip install -U pip setuptools wheel   und    pip install spacy 
#de_core_news_sm-3.5.0-py3-none-any.whl (muss gedownloaded werden -> https://spacy.io/models/de/)
#de_core_news_lg-3.5.0-py3-none-any.whl
#de_dep_news_trf-3.5.0-py3-none-any.whl

nlp = spacy.load("de_core_news_sm")
#nlp1 = spacy.load ("de_dep_news_trf")
nlp2 = spacy.load ("de_core_news_lg")

list_of_nouns, list_of_pronouns = [], []

def spacify_with_coref(lyrics_from_song):
    doc3 = nlp2(lyrics_from_song)
    doc = nlp(lyrics_from_song)

    doc._.coref_chains = doc3._.coref_chains

    return doc

for song_id in tqdm(range(len(songs_dict["Prototyp"]))):

    lyrics_from_song = songs_dict["Prototyp"][str(song_id)]["lyrics"]
    doc = nlp(lyrics_from_song)
    #doc2 = nlp1 (lyrics_from_song)
    doc3 = nlp2 (lyrics_from_song)

    # Ausgabe lediglich von Wörtern, die Nomen und Pronomen sind:
    for token in doc3: 
        if token.pos_ == "NOUN":
            list_of_nouns.append(token.text)
        elif token.pos_ == "PRON":
            list_of_pronouns.append(token.text)
        else:
            pass

100%|██████████| 88/88 [00:20<00:00,  4.27it/s]


In [33]:
print(f"{len(list_of_nouns)} nouns were found.")
print(f"{len(list_of_pronouns)} pronouns were found.")

6516 nouns were found.
5504 pronouns were found.


In [34]:
distinct_nouns = sorted(list(set(list_of_nouns)))
distinct_pronouns = sorted(list(set(list_of_pronouns)))
print(f"{len(distinct_nouns)} distinct nouns were found.")
print(f"{len(distinct_pronouns)} distinct pronouns were found.")

2169 distinct nouns were found.
110 distinct pronouns were found.


2. Daten sortieren



NOMEN

Json-Datei wird erstellt, um eine Liste mit allen Nomen und zugehörigen Artikeln zu bekommen. (Durchlaufzeit 28 min)

In [ ]:
scrape_data = False # decide, whether to scrape full data again or read an exisiting json file.

In [ ]:
if scrape_data:
    # Dictionary
    result_nomen = {"Nomen": {}}

    unknown_nomen = 0
    id = 0
    for word in tqdm(list_of_nouns):
        n = duden.get(word)
        if n is not None:
            result_nomen["Nomen"][id] = {"WORT": n.name, "ARTIKEL": n.article}
            id += 1
        else:
            unknown_nomen += 1

    with open("Nomen.json", "w") as fp:
        dict_result_nomen = json.dump(result_nomen, fp, indent=4)
else:
    with open("./Nomen.json", "r") as f:
        result_nomen = json.load(f)

Einzelne Json-Dateien für Nomen (m,w,n) erstellen.

In [38]:
nomen_m, nomen_w, nomen_n = [], [], []

for id in tqdm(result_nomen["Nomen"]):
    if result_nomen["Nomen"][id]["ARTIKEL"] == "der":
        nomen_m.append(result_nomen["Nomen"][id]["WORT"])
    elif result_nomen["Nomen"][id]["ARTIKEL"] == "die":
        nomen_w.append(result_nomen["Nomen"][id]["WORT"])
    elif result_nomen["Nomen"][id]["ARTIKEL"] == "das":
        nomen_n.append(result_nomen["Nomen"][id]["WORT"])
    else:
        pass

nomen_sortiert = {"Männlich": nomen_m, "Weiblich": nomen_w, "Neutral": nomen_n}

with open("Nomen_sortiert.json", "w") as fp:
    json.dump(nomen_sortiert, fp, indent=4)

100%|██████████| 3205/3205 [00:00<00:00, 832419.61it/s]


PRONOMEN

Pronomen nach männlich und weiblich unterteilen.

In [144]:
pronomen_m, pronomen_w = [],[]

for word in list_of_pronouns:
    #männlich
    #personal
    if word == "er":
        pronomen_m.append(word)
    elif word == "Er":
        pronomen_m.append(word)
    elif word == "ER":
        pronomen_m.append(word)
    elif word == "ihn":
        pronomen_m.append(word)
    elif word == "Ihn":
        pronomen_m.append(word)
    elif word == "IHN":
        pronomen_m.append(word)
    elif word == "ihm":
        pronomen_m.append(word)
    elif word == "Ihm":
        pronomen_m.append(word)
    elif word == "IHM":
        pronomen_m.append(word)
    #possesiv
    elif word == "sein":
        pronomen_m.append(word)
    elif word == "Sein":
        pronomen_m.append(word)
    elif word == "SEIN":
        pronomen_m.append(word)
    elif word == "seine":
        pronomen_m.append(word)
    elif word == "Seine":
        pronomen_m.append(word)
    elif word == "SEINE":
        pronomen_m.append(word)
    elif word == "Seiner":
        pronomen_m.append(word)
    elif word == "seiner":
        pronomen_m.append(word)
    elif word == "SEINER":
        pronomen_m.append(word)
    elif word == "Seines":
        pronomen_m.append(word)
    elif word == "seines":
        pronomen_m.append(word)
    elif word == "SEINES":
        pronomen_m.append(word)
    elif word == "Seinen":
        pronomen_m.append(word)
    elif word == "seinen":
        pronomen_m.append(word)
    elif word == "SEINEN":
        pronomen_m.append(word)
    elif word == "Seinem":
        pronomen_m.append(word)
    elif word == "seinem":
        pronomen_m.append(word)
    elif word == "SEINEM":
        pronomen_m.append(word)
    #demonstrativ
    elif word == "Der":
        pronomen_m.append(word)
    elif word == "der":
        pronomen_m.append(word)
    elif word == "DER":
        pronomen_m.append(word)
    elif word == "Dieser":
        pronomen_m.append(word)
    elif word == "DIESER":
        pronomen_m.append(word)
    elif word == "dieser":
        pronomen_m.append(word)
    elif word == "Derjenige":
        pronomen_m.append(word)
    elif word == "DERJENIGE":
        pronomen_m.append(word)
    elif word == "derjenige":
        pronomen_m.append(word)
    elif word == "Derselbe":
        pronomen_m.append(word)
    elif word == "DERSELBE":
        pronomen_m.append(word)
    elif word == "derselbe":
        pronomen_m.append(word)
    elif word == "Jener":
        pronomen_m.append(word)
    elif word == "jener":
        pronomen_m.append(word)
    elif word == "JENER":
        pronomen_m.append(word)
    #weiblich
    #personal
    if word == "sie":
        pronomen_w.append(word)
    elif word == "Sie":
        pronomen_w.append(word)
    elif word == "SIE":
        pronomen_w.append(word)
    elif word == "ihr":
        pronomen_w.append(word)
    elif word == "IHR":
        pronomen_w.append(word)
    elif word == "Ihr":
        pronomen_w.append(word)
    #possessiv
    elif word == "Ihre":
        pronomen_w.append(word)
    elif word == "IHRE":
        pronomen_w.append(word)
    elif word == "ihre":
        pronomen_w.append(word)
    elif word == "IHREN":
        pronomen_w.append(word)
    elif word == "Ihren":
        pronomen_w.append(word)
    elif word == "ihren":
        pronomen_w.append(word)
    elif word == "IHREM":
        pronomen_w.append(word)
    elif word == "ihrem":
        pronomen_w.append(word)
    elif word == "Ihrem":
        pronomen_w.append(word)
    #demonstrativ
    elif word == "Die":
        pronomen_w.append(word)
    elif word == "DIE":
        pronomen_w.append(word)
    elif word == "die":
        pronomen_w.append(word)
    elif word == "Diejenige":
        pronomen_w.append(word)
    elif word == "diejenige":
        pronomen_w.append(word)
    elif word == "DIEJENIGE":
        pronomen_w.append(word)
    elif word == "DIESELBE":
        pronomen_w.append(word)
    elif word == "dieselbe":
        pronomen_w.append(word)
    elif word == "Dieselbe":
        pronomen_w.append(word)
    elif word == "DIESE":
        pronomen_w.append(word)
    elif word == "diese":
        pronomen_w.append(word)
    elif word == "Diese":
        pronomen_w.append(word)
    elif word == "Jene":
        pronomen_w.append(word)
    elif word == "JENE":
        pronomen_w.append(word)
    elif word == "jene":
        pronomen_w.append(word)
   
    else:
        pass

with open('Pronomen_m.json', 'w') as fp:
      dict_result_nomen=json.dump(pronomen_m, fp, indent=4)
with open('Pronomen_w.json', 'w') as fp:
      dict_result_nomen=json.dump(pronomen_w, fp, indent=4)


['er', 'der', 'Er', 'er', 'Er', 'er', 'er', 'ihn', 'ihm', 'der', 'er', 'er', 'der', 'er', 'er', 'der', 'der', 'er', 'der', 'der', 'ihn', 'ihn', 'ihn', 'der', 'ihn', 'der', 'der', 'der', 'der', 'der', 'der', 'der', 'ihn', 'ihn', 'ihn', 'ihm', 'Er', 'Er', 'er', 'er', 'er', 'er', 'er', 'er', 'er', 'er', 'er', 'der', 'der', 'ihn', 'ihn', 'ihn', 'ihn', 'er', 'er', 'er', 'er', 'dieser', 'dieser', 'dieser', 'ihn', 'er', 'er', 'ihn', 'ihm', 'ihm', 'der', 'der', 'der', 'der', 'der']
['Sie', 'Sie', 'Sie', 'sie', 'sie', 'Sie', 'sie', 'sie', 'die', 'sie', 'sie', 'Die', 'die', 'sie', 'sie', 'sie', 'sie', 'sie', 'Sie', 'Sie', 'Sie', 'sie', 'sie', 'Sie', 'sie', 'sie', 'sie', 'die', 'sie', 'die', 'sie', 'sie', 'sie', 'sie', 'sie', 'sie', 'Sie', 'sie', 'sie', 'sie', 'ihr', 'die', 'die', 'die', 'sie', 'sie', 'sie', 'die', 'die', 'sie', 'Sie', 'sie', 'sie', 'sie', 'sie', 'Sie', 'Sie', 'Sie', 'Sie', 'sie', 'sie', 'sie', 'sie', 'sie', 'Sie', 'sie', 'Sie', 'Die', 'Die', 'sie', 'sie', 'sie', 'die', 'die', 'd

In [1]:
if word.lower() in ["er", "ihn", "ihm"]:
    pronomen_m.append(word)

'abc'

# Title tbd

- Sonderzeichen in strings?

### Read files

In [39]:
gender_neu = pd.read_csv("./gender_neu_Nomen.csv",sep=";",header=None,encoding="utf8")
gender_neu = gender_neu.iloc[:, 0]
gender_neu.columns = ["Wort"]
gender_neu = list(gender_neu)

with open("./Nomen_sortiert.json", "r") as f:
    nomen_sortiert = json.load(f)

In [48]:
complete_df = pd.DataFrame(columns=["Geschlecht","Wort","neutral"])
tmp = []
for g in nomen_sortiert.keys(): # geschlecht
    for w in nomen_sortiert[g]: # wort
        is_gender_neutral = True if w in gender_neu else False

        complete_df.loc[len(complete_df)] = [g, w, is_gender_neutral]
        tmp.append(w)


In [50]:
freq = Counter(tmp)

In [52]:
freq

Counter({'Refrain': 136,
         'Strophe': 103,
         'Zeit': 99,
         'Welt': 96,
         'Kopf': 91,
         'Liebe': 89,
         'Leben': 78,
         'Nacht': 48,
         'Herz': 47,
         'Part': 44,
         'Sommer': 41,
         'Songtext': 38,
         'Oh': 33,
         'Outro': 30,
         'Himmel': 29,
         'Bauch': 27,
         'Stadt': 27,
         'Hand': 26,
         'Baby': 26,
         'Ende': 25,
         'Welle': 23,
         'Augenblick': 22,
         'Stimme': 22,
         'Sonne': 21,
         'Haus': 20,
         'Halt': 18,
         'Geld': 18,
         'Lied': 18,
         'Lichter': 17,
         'Frau': 17,
         'Musik': 17,
         'Applaus': 16,
         'Astronaut': 15,
         'Finger': 15,
         'Blut': 15,
         'Block': 14,
         'Raum': 13,
         'Angst': 13,
         'Licht': 13,
         'Lieblingsmensch': 12,
         'Schmerz': 12,
         'Wolke': 12,
         'Spiel': 12,
         'Rest': 11,
         'Weg

In [44]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [53]:
complete_df.loc[complete_df["neutral"] == True, :]

,Geschlecht,Wort,neutral
30,Männlich,Vater,True
88,Männlich,Lieblingsmensch,True
89,Männlich,Lieblingsmensch,True
90,Männlich,Lieblingsmensch,True
91,Männlich,Lieblingsmensch,True
92,Männlich,Kreis,True
95,Männlich,Lieblingsmensch,True
96,Männlich,Lieblingsmensch,True
97,Männlich,Lieblingsmensch,True
98,Männlich,Lieblingsmensch,True


In [22]:
nomen_sortiert

{'Männlich': ['Abend', 'Refrain', 'Tresen', 'Refrain', 'Schnaps', 'Refrain'],
 'Weiblich': ['Kneipe',
  'Ecke',
  'Liebe',
  'Freundschaft',
  'Stadt',
  'Wand',
  'Kneipe',
  'Ecke',
  'Kneipe',
  'Ecke',
  'Uhr'],
 'Neutral': ['Leben', 'Dorf', 'Leben', 'Licht', 'Licht', 'Licht']}